In [ ]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
import logging
import os

os.makedirs('logs', exist_ok=True)
logging.basicConfig(filename='./logs/Illuminate_operations.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
logging.info('\n\n-------------New Illuminate Operations Logging Instance')


access_token, expires_in = get_access_token()

#For testing
assessments_df, assessment_id_list = get_all_assessments_metadata(access_token)
missing_ids_from_metadata = ['114845', '141498'] # Add assessments that are not present in assessements metadata
assessment_id_list = list(set(assessment_id_list + missing_ids_from_metadata))


test_results_group, log_results_group = loop_through_assessment_scores(access_token, assessment_id_list, 'Group') #Seperate table
test_results_standard, log_results_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'Standard')
test_results_no_standard, log_results_no_standard = loop_through_assessment_scores(access_token, assessment_id_list, 'No_Standard')

test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
test_results_view = create_test_results_view(test_results_combined, '23-24') #add in grade level col, string matching

# Testing View Changes

In [61]:
%load_ext autoreload
%autoreload 2

from modules.auth import *
from modules.assessments_endpoints import *
from modules.frame_transformations import *
from modules.config import base_url_illuminate
import logging
import os

test_results_combined = bring_together_test_results(test_results_no_standard, test_results_standard)
test_results_view = create_test_results_view(test_results_combined, '23-24') #add in grade level col, string matching

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read in Raw Files

In [ ]:
import pandas as pd
c =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_combined.csv')
v =pd.read_csv('/home/g2015samtaylor/illuminate/assessment_results_view.csv')
sr =pd.read_table('/home/icef/powerschool/Student_Rosters.txt')